In [10]:
from __future__ import unicode_literals

from collections import defaultdict
import srsly

import os
import pandas as pd

from spacy.errors import Errors
from spacy.compat import basestring_
from spacy.util import ensure_path
from spacy.tokens import Span
from spacy.matcher import Matcher, PhraseMatcher

from spacy import displacy

class EntityMatcher(object):
    name = "entity_matcher"

    def __init__(self, nlp,**cfg):
        self.nlp = nlp
        self.overwrite = cfg.get("overwrite_ents", False)
        self.token_patterns = defaultdict(list)
        self.phrase_patterns = defaultdict(list)
        self.matcher = Matcher(nlp.vocab)
        self.phrase_matcher = PhraseMatcher(nlp.vocab)

        patterns = cfg.get("patterns")
        if patterns is not None:
            self.add_patterns(patterns)
            
            
    def __len__(self):
        """The number of all patterns added to the entity ruler."""
        n_token_patterns = sum(len(p) for p in self.token_patterns.values())
        n_phrase_patterns = sum(len(p) for p in self.phrase_patterns.values())
        return n_token_patterns + n_phrase_patterns
    
    def __contains__(self, label):
        """Whether a label is present in the patterns."""
        return label in self.token_patterns or label in self.phrase_patterns

    def __call__(self, doc):
        """Find matches in document and add them as entities.

        doc (Doc): The Doc object in the pipeline.
        RETURNS (Doc): The Doc with added entities, if available.

        DOCS: https://spacy.io/api/entityruler#call
        """
        
        matches = list(self.matcher(doc)) + list(self.phrase_matcher(doc))
        
        matches = set(
            [(m_id, start, end) for m_id, start, end in matches if start != end]
        )
        get_sort_key = lambda m: (m[2] - m[1], m[1])
        matches = sorted(matches, key=get_sort_key, reverse=False)
        
        
        entities = list(doc.ents)
        new_entities = []
        seen_tokens = set()
        for match_id, start, end in matches:
            
            if any(t.ent_type for t in doc[start:end]) and not self.overwrite:
                continue
            
            if start not in seen_tokens and end - 1 not in seen_tokens:

                new_entities.append(Span(doc, start, end, label=match_id))

                entities = [e for e in entities if not (e.start < end and e.end > start)]
                seen_tokens.update(range(start, end))
                
        doc.ents = entities + new_entities

        return doc

    @property
    def labels(self):
        """All labels present in the match patterns.

        RETURNS (set): The string labels.

        DOCS: https://spacy.io/api/entityruler#labels
        """
        all_labels = set(self.token_patterns.keys())
        all_labels.update(self.phrase_patterns.keys())
        return tuple(all_labels)

    @property
    def patterns(self):
        """Get all patterns that were added to the entity ruler.

        RETURNS (list): The original patterns, one dictionary per pattern.

        DOCS: https://spacy.io/api/entityruler#patterns
        """
        all_patterns = []
        for label, patterns in self.token_patterns.items():
            for pattern in patterns:
                all_patterns.append({"label": label, "pattern": pattern})
        for label, patterns in self.phrase_patterns.items():
            for pattern in patterns:
                all_patterns.append({"label": label, "pattern": pattern.text})
        return all_patterns

    def add_patterns(self, patterns):
        """Add patterns to the entitiy ruler. A pattern can either be a token
        pattern (list of dicts) or a phrase pattern (string). For example:
        {'label': 'ORG', 'pattern': 'Apple'}
        {'label': 'GPE', 'pattern': [{'lower': 'san'}, {'lower': 'francisco'}]}

        patterns (list): The patterns to add.

        DOCS: https://spacy.io/api/entityruler#add_patterns
        """
        for entry in patterns:
            label = entry["label"]
            pattern = entry["pattern"]
            on_match = entry['on_match']
            
            if on_match == 'None':
                on_matcher = None
            else:

                print(label)
                def on_matcher(matcher, doc, id, matches):
                    match_id, start, end = matches[-1]
                    print('This is the on match[0]:')
                    print(on_match[0])
                    for callback in on_match:
                        print('This is the callback:')
                        print(callback)
                        if 'TRUNCR' in callback.keys():
                            end = end - callback['TRUNCR']
                            matches[id] = (match_id, start, end)

                    
                        if 'TRUNCL' in callback.keys():
                            start = start + callback['TRUNCL']
                            matches[id] = (match_id, start, end)
                            
#                    matches.append(('sub-CLUE',start - callback['TRUNCL'],start))
                    print(matches)
                        

            if isinstance(pattern, basestring_):
                self.phrase_patterns[label].append(self.nlp(pattern))
            elif isinstance(pattern, list):
                self.token_patterns[label].append((on_matcher,pattern))

            else:
                raise ValueError(Errors.E097.format(pattern=pattern))
                


        for label, match in self.token_patterns.items():
            for on_matcher,pattern in match:
                self.matcher.add(label, on_matcher, pattern)
        for label, patterns in self.phrase_patterns.items():
            self.phrase_matcher.add(label, None, *patterns)

    def from_bytes(self, patterns_bytes, **kwargs):
        """Load the entity ruler from a bytestring.

        patterns_bytes (bytes): The bytestring to load.
        **kwargs: Other config paramters, mostly for consistency.
        RETURNS (EntityRuler): The loaded entity ruler.

        DOCS: https://spacy.io/api/entityruler#from_bytes
        """
        patterns = srsly.msgpack_loads(patterns_bytes)
        self.add_patterns(patterns)
        return self

    def to_bytes(self, **kwargs):
        """Serialize the entity ruler patterns to a bytestring.

        RETURNS (bytes): The serialized patterns.

        DOCS: https://spacy.io/api/entityruler#to_bytes
        """
        return srsly.msgpack_dumps(self.patterns)

    def from_disk(self, path, **kwargs):
        """Load the entity ruler from a file. Expects a file containing
        newline-delimited JSON (JSONL) with one entry per line.

        path (unicode / Path): The JSONL file to load.
        **kwargs: Other config paramters, mostly for consistency.
        RETURNS (EntityRuler): The loaded entity ruler.

        DOCS: https://spacy.io/api/entityruler#from_disk
        """
        path = ensure_path(path)
        path = path.with_suffix(".jsonl")
        patterns = srsly.read_jsonl(path)
        self.add_patterns(patterns)
        return self

    def to_disk(self, path, **kwargs):
        """Save the entity ruler patterns to a directory. The patterns will be
        saved as newline-delimited JSON (JSONL).

        path (unicode / Path): The JSONL file to load.
        **kwargs: Other config paramters, mostly for consistency.
        RETURNS (EntityRuler): The loaded entity ruler.

        DOCS: https://spacy.io/api/entityruler#to_disk
        """
        path = ensure_path(path)
        path = path.with_suffix(".jsonl")
        srsly.write_jsonl(path, self.patterns)

        
def render_doc(doc, entity = False):
    '''This function render the text of the documents with the entity signed'''
    
    out = displacy.parse_ents(doc)
    print(out)
    adder = 0
    
    for i,ent in enumerate(out['ents']):
                
        #do not add the first entity match
        out['ents'][i]['start'] += i*adder
        out['ents'][i]['end'] += i*adder
        
        if entity == False:
            adder = 4
            if ent['label'] == 'sub-CLUE':
                out['text'] = out['text'][:ent['start']] + '_*' + out['text'][ent['start']:ent['end']] + '*_' + out['text'][ent['end']:]
            else:
                out['text'] = out['text'][:ent['start']] + '**' + out['text'][ent['start']:ent['end']] + '**' + out['text'][ent['end']:]
            
            
        else:
            adder = len(ent['label']) + 6
            if ent['label'] == 'sub-CLUE':
                out['text'] = out['text'][:ent['start']] + '_*' + out['text'][ent['start']:ent['end']] + '*_{' + ent['label'] + '}' + out['text'][ent['end']:]
            else:
                out['text'] = out['text'][:ent['start']] + '**' + out['text'][ent['start']:ent['end']] + '**{' + ent['label'] + '}' + out['text'][ent['end']:]
            
        
    return out['text']

In [11]:
xls = pd.ExcelFile('../15. Cluster SS-ESCO/data/input/ulisseDB.xlsx')

for sheet in xls.sheet_names:
    globals()[sheet] = xls.parse(sheet_name = sheet)

In [4]:
import spacy
from spacy.lang.en import English

nlp = English()
nlp = spacy.load("it_core_news_sm")


entity_matcher = EntityMatcher(nlp).from_disk("gestione.jsonl")
nlp.add_pipe(entity_matcher)


#TEXTS = Abstracts.abstract.tolist()

for text in TEXTS:
    doc = nlp(text)
    print([ent.label_ for ent in doc.ents if (ent.label_ == 'PROBLEM SOLVING')])

In [19]:
doc

Â© The Author(s) 2016. The aim of this study was to evaluate the impact of a social contact and education intervention to improve attitudes to mental illness in first-year social work students. This was a 3-month cluster randomized controlled trial with two parallel arms: intervention (87) and control group (79). The intervention was a workshop led by an OBERTAMENT activist (a person with a mental illness trained in communication skills and empowerment by a social worker). We assessed intended future behavior toward people with mental illness, personal and perceived stigma, and mental healthâ€“related attitudes (self-reported questionnaire). The intervention improved social work studentsâ€™ attitudes (d ÍŒ 0.50, p <.05) and reduced personal stigma toward people with mental illness (d = 0.35, p = 04) as well as improving their future intended behavior 2 weeks after the intervention (d = 0.51, p =.01). The intervention impact on authoritarian attitudes toward people with schizophrenia wa